Updated:   
1. Исправлена логика построения списка исключений;  
2. Добавлены правила капитализации: "God", заглавная после 2 заглавных;   
3. Для подсчёта длины арифметического кода добавлена длина передаваемой статистики.   
  
Степень сжатия отличается от энтропии на 4-ом знаке    
Размер бинарной маски исключений стал в 2.6 раз меньше  

# Декапитализация

Возьмём текст "Критика чистого разума" Иммануила Канта (text.txt) и применим к нему build_exceptions, строющую исключения их стандартных правил капитализации.

In [ ]:
import re

def build_exceptions(text):
    should_be_uppercase = [0] * len(text)

    # Правило 1: первая буква текста
    for i, c in enumerate(text):
        if c.isalpha():
            should_be_uppercase[i] = 1
            break

    # Правило 2: после [.?!] + пробелы/перевод строки/кавычки/скобки
    pattern = re.compile(r'[\.\!\?][\'\"\)\]\}]*[\s\n]+([A-Za-z])')
    for match in pattern.finditer(text):
        idx = match.start(1)
        should_be_uppercase[idx] = 1

    # Правило 3: одиночная I
    for match in re.finditer(r'\bI\b', text):
        idx = match.start()
        should_be_uppercase[idx] = 1


    # Правило 4: Имена, которые всегда с заглавной буквы
    special_words = ['God']

    for word in special_words:
        pattern = re.compile(rf'\b{re.escape(word)}\b', flags=re.IGNORECASE)
        for match in pattern.finditer(text):
            idx = match.start()
            # Отмечаем первую букву как "должна быть заглавной"
            should_be_uppercase[idx] = 1

    # Правило 5: после двух заглавных
    uppercase_streak = 0
    for i, c in enumerate(text):
        if c.isupper():
            uppercase_streak += 1
        elif c == ' ':
            pass
        else:
            # Если встречается любой не пробел и не заглавная, сбрасываем счётчик
            uppercase_streak = 0

        # Если >= 2 заглавных до этой буквы, ожидаем, что текущая буква должна быть заглавной
        if c.isalpha() and uppercase_streak >= 2:
            should_be_uppercase[i] = 1

    # Строим битовый массив исключений
    exceptions = [0] * len(text)
    for i, c in enumerate(text):
        if c.isalpha():
            expected_upper = should_be_uppercase[i]
            actual_upper = int(c.isupper())
            if expected_upper != actual_upper:
                exceptions[i] = 1

    return exceptions

def decapitalize(text):
    return ''.join(c.lower() if c.isupper() else c for c in text)

# Чтение файла
filename = 'text.txt'
with open(filename, 'r', encoding='ASCII') as f:
    text = f.read()

exceptions = build_exceptions(text)
lower_text = decapitalize(text)

# Сохранение результатов
with open('decapitalized.txt', 'w', encoding='ASCII') as f:
    f.write(lower_text)

with open('exceptions.txt', 'w', encoding='ASCII') as f:
    f.write(''.join(str(j) for j in exceptions))


Использованные правила капитализации:
1. Стандартные (Первая буква текста, одиночная I, после .!?);
2. Слова, которе всегда пишутся с заглавной буквы ('God');
3. Заглавная после двух заглавных подряд.

In [3]:
filename = 'text.txt'
with open(filename, 'r', encoding='ASCII') as f:
    text = f.read()

In [4]:
print(f'Символов в тексте: {len(text)}')

uppercase_count = sum(1 for char in text if char.isupper())
print(f"Количество заглавных букв в тексте: {uppercase_count}")

Символов в тексте: 1268316
Количество заглавных букв в тексте: 10131


In [5]:
filename = 'exceptions.txt'
with open(filename, 'r', encoding='ASCII') as f:
    exceptions_text = f.read()

Посмотрим на исключения:

Для проверки того, что 'God' не записывается в исключения можно использовать  
letter_from = 45915  
letter_to = 50000  

In [6]:
letter_from = 0
letter_to = 10_000

print(''.join([text[i] if exceptions_text[i] == '1' else ' ' for i in range(letter_from, letter_to)]))
print(''.join([text[i] if text[i] != '\n' else ' ' for i in range(letter_from, letter_to)]))

                                                                                                                                      I        K                                          J                                P                                     H                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [24]:
exceptions_amount = sum(exceptions)

print(f'Всего символов: {len(exceptions)}')
print(f'Всего исключений: {exceptions_amount}')
percent_1 = exceptions_amount / len(exceptions)
print(f'Доля исключений из всего текста: {round(percent_1 * 100, 3)}%')

print(f'Число объяснённых заглавных букв правилами капитализации = {uppercase_count - exceptions_amount}')
print(f'Доля объяснённый заглавных букв правилами капитализации = {round((uppercase_count - exceptions_amount) / uppercase_count * 100, 2)}%')

Всего символов: 1268316
Всего исключений: 1569
Доля исключений из всего текста: 0.124%
Число объяснённых заглавных букв правилами капитализации = 8562
Доля объяснённый заглавных букв правилами капитализации = 84.51%


In [25]:
import math
entropy = -(math.log2(percent_1)*percent_1 + math.log2(1-percent_1)*(1-percent_1))
print(f'Энтропия = {round(entropy, 7)} \n')

print(f'Если передать позиции {exceptions_amount} исключений, зашифровав в 21 бит каждое, то получится {exceptions_amount * 21} бит или {round(exceptions_amount * 21 / 8 / 1024, 2)} кб')
print(f'Тогда степень сжатия = {round(exceptions_amount * 21 / len(exceptions), 5)} => baseline = {round(exceptions_amount * 21 / len(exceptions) * 100, 2)}% сжатия битовой маски')

Энтропия = 0.0137323 

Если передать позиции 1569 исключений, зашифровав в 21 бит каждое, то получится 32949 бит или 4.02 кб
Тогда степень сжатия = 0.02598 => baseline = 2.6% сжатия битовой маски


# Арифметическое кодирование


Алгоритм взят отсюда: https://github.com/tommyod/arithmetic-coding 

In [26]:
from arithmetic_coding import ArithmeticEncoder


message = [str(i) for i in exceptions] + ['<EOM>']
frequencies = {'0': len(exceptions) - exceptions_amount, '1': exceptions_amount, '<EOM>':1}
encoder = ArithmeticEncoder(frequencies=frequencies, bits=128)
bits = list(encoder.encode(message))
bits[:10]

[1, 1, 0, 1, 1, 0, 0, 0, 1, 1]

Декодеру ещё необходимо передать:
1. Количество имён (символов), частоты которых считали
1. Список имён, частоты которых считали
2. Их частоты
3. Список имён - исключений ('God')

In [27]:
def compute_total_encoded_size(bits, frequencies, names_list):
    # Длина основного сообщения
    message_bits_len = len(bits)

    # Длина передаваемой таблицы частот
    num_symbols_bits = 8  # 1 байт для передачи количества символов

    per_symbol_bits = 0
    for symbol, freq in frequencies.items():
        symbol_bytes = symbol.encode('ASCII')
        symbol_length = len(symbol_bytes)

        symbol_bits = symbol_length * 8  # всего байт на кодируемое имя (<EOM> - 5 байт)

        # На частоту:
        freq_bits = 32  # 4 байта на частоту

        per_symbol_bits += symbol_bits + freq_bits

    total_freq_bits = num_symbols_bits + per_symbol_bits

    # Длина передаваемого списка имён
    # Считаем, что имена кодируются как ASCII + b'\x00' после каждого имени
    delimiter = b'\x00'
    names_bytes = bytearray()
    for s in names_list:
        names_bytes.extend(s.encode('ASCII') + delimiter)

    names_bits_len = len(names_bytes) * 8

    # Общая длина
    total_bits = message_bits_len + total_freq_bits + names_bits_len

    # Выводим подробности
    print(f'Длина основного сообщения: {message_bits_len} бит')
    print(f'Длина таблицы частот: {total_freq_bits} бит')
    print(f'Длина списка имён: {names_bits_len} бит')
    print(f'Общая длина: {total_bits} бит')

    return total_bits


In [31]:
total_lenght = compute_total_encoded_size(bits, frequencies, ['God'])


Длина основного сообщения: 17440 бит
Длина таблицы частот: 160 бит
Длина списка имён: 32 бит
Общая длина: 17632 бит


In [ ]:
print(f'Длина получившейся битовой строки = {total_lenght} бит или {round(total_lenght / 8 / 1024, 3)} кб')
print(f'Что составляет {round(total_lenght / len(exceptions), 5)} от длины всей маски исключений\n')

print(f'Степень сжатия больше энтропии на {round(total_lenght / len(exceptions) - entropy, 5)}')
print(f'Полученный результат уже достаточно хорош, чтобы мы могли выбрать арифметическое кодирование')

Длина получившейся битовой строки = 17632 бит или 2.152 кб
Что составляет 0.0139 от длины всей маски исключений

Степень сжатия больше энтропии на 0.00017
Полученный результат уже достаточно хорош, чтобы мы могли выбрать арифметическое кодирование
